<a href="https://colab.research.google.com/github/abasilio91/aulas_ANN_ufu/blob/main/projects/ann_article_settling_reproduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>